## Automated Sleep Labeler

Use this notebook to make automated sleep labels.

Open this notebook in the terminal:

```bash
$ jupyter notebook

```

Then edit these constants:

In [ ]:
# File paths and parameters
WINDOW_DURATION = 2.0 # in seconds
LFP_DIRECTORY = '/'
LFP_FILE_PREFIX = 'Mouse4'

And then run the rest of these cells:

In [35]:
from bokeh.models.callbacks import CustomJS
from bokeh.io import show, output_notebook
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Button
from bokeh.plotting import figure
from bokeh.themes import Theme
from matplotlib.colors import to_hex
import numpy as np
import os
import yaml

import lpne

output_notebook()

Loading BokehJS ...

In [28]:
# NOTE: test this!
lfp_fns = lpne.get_lfp_filenames(LFP_DIR)
lfp_fns = [i for i in lfp_fns if i.startswith(LFP_FILE_PREFIX)]

In [ ]:
# 

def 
idx = 0
for fn, out_fn in zip(FNS, OUT_FNS):
    lines = []
    t1, t2 = np.loadtxt(fn, usecols=(0,1), unpack=True)
    prev_label, start_t = None, None
    for j in range(len(t1)):
        label = labels[colors.index(cs[idx])]
        if label != prev_label:
            if j > 0 and prev_label != '_':
                line = f"{start_t:.6f}\t{t1[j]:.6f}\t{prev_label}\n"
                lines.append(line)
            start_t = t1[j]
            prev_label = label
        if j == len(t1)-1 and prev_label != '_':
            line = f"{start_t:.6f}\t{t1[j]:.6f}\t{prev_label}\n"
            lines.append(line)
        idx += 1

    with open(out_fn, 'w+') as f:
        f.writelines(lines)

print("Done.")

In [33]:
# App-related constants
BUTTON_SIZE = 200
TOOLS = 'lasso_select,pan,wheel_zoom,reset'
WAKE_COLOR = to_hex('dodgerblue')
NREM_COLOR = to_hex('mediumseagreen')
REM_COLOR = to_hex('darkorchid')
UNLABELED_COLOR = to_hex('peru')


def my_app(doc):
    """Sleep labeling app."""    
    source = ColumnDataSource()
    plot = figure(tools=TOOLS)
    plot.circle(
            x="x",
            y="y",
            source=source,
            size=8,
            color="color",
            line_color=None,
    )
    
    wake_button = Button(label="Wake", default_size=BUTTON_SIZE)
    nrem_button = Button(label="NREM", default_size=BUTTON_SIZE)
    rem_button = Button(label="REM", default_size=BUTTON_SIZE)
    unlabeled_button = Button(label="Unlabeled", default_size=BUTTON_SIZE)
    save_button = Button(label="Save", default_size=200)
    
    
    wake_button.js_on_click(
        CustomJS(
            args=dict(source=source),
            code=f"""
                var color = source.data['color']
                var idx = source.selected.indices;
                var selected_length = idx.length;
                for (var i=0; i<selected_length; i++) {{
                    color[idx[i]] = '{WAKE_COLOR}';
                }}
                source.selected.indices = [];
                source.change.emit();
            """,
    ))
    nrem_button.js_on_click(
        CustomJS(
            args=dict(source=source),
            code=f"""
                var color = source.data['color']
                var idx = source.selected.indices;
                var selected_length = idx.length;
                for (var i=0; i<selected_length; i++) {{
                    color[idx[i]] = '{NREM_COLOR}';
                }}
                source.selected.indices = [];
                source.change.emit();
            """,
    ))
    rem_button.js_on_click(
        CustomJS(
            args=dict(source=source),
            code=f"""
                var color = source.data['color']
                var idx = source.selected.indices;
                var selected_length = idx.length;
                for (var i=0; i<selected_length; i++) {{
                    color[idx[i]] = '{REM_COLOR}';
                }}
                source.selected.indices = [];
                source.change.emit();
            """,
    ))
    unlabeled_button.js_on_click(
        CustomJS(
            args=dict(source=source),
            code=f"""
                var color = source.data['color']
                var idx = source.selected.indices;
                var selected_length = idx.length;
                for (var i=0; i<selected_length; i++) {{
                    color[idx[i]] = '{UNLABELED_COLOR}';
                }}
                source.selected.indices = [];
                source.change.emit();
            """,
    ))
    
    
    def save_callback():
        # Save the labels here...
        source.data['color'] = ['yellow']*5
    
    
    save_button.on_click(save_callback)
    
    x = [1,2,3,4,5]
    y = [5,5,4,6,2]
    color = [UNLABELED_COLOR]*len(x)
    source.data = dict(
        x=x,
        y=y,
        color=color,
    )
    
    buttons = column(
            wake_button,
            nrem_button,
            rem_button,
            unlabeled_button,
            save_button,
    )
    doc.add_root(row(buttons, plot))
    
    doc.theme = Theme(json=yaml.load("""
        attrs:
            Figure:
                background_fill_color: "#EEEEEE"
                outline_line_color: white
                toolbar_location: above
                height: 500
                width: 800
            Grid:
                grid_line_dash: [6, 4]
                grid_line_color: white
    """, Loader=yaml.FullLoader))

In [34]:
show(my_app)